In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [2]:
from evallm.experiments.sequence_completion_summary import (
    sequence_completion_results,
    display_prompt as display_prompt_sc,
)

from evallm.experiments.transducer_summary import (
    transducer_results,
)
from evallm.experiments.main_tables import (
    best_prompt,
    multi_prompts,
    main_table_of_results,
    multi_prompt_table_of_results,
)

from evallm.experiments.transducer_plotting import produce_table
from evallm.experiments.models_display import model_by_display_key

In [3]:
results_t = transducer_results()
results_sc = sequence_completion_results()

### Main Table

In [4]:
main_table_of_results(best_prompt(results_t), best_prompt(results_sc))

\begin{tabular}{|l|c|c|c|c|c|c|c|c|}
\hline
\bf Model & \bf Size & \bf IT? & \bf Code? & \bf Sequence Completion & \bf SR & \bf Transducer & \bf TR\\
\hline
\multicolumn{8}{|c|}{ \bf Baselines} \\
\hline
\textsc{BruteForce} & -- &  &  & \cellcolor{lightgray} 100.0 (99.9--100.0) & 1 & \cellcolor{lightgray} 96.4 (96.2--96.7) & 1 \\
\hline
6-\textsc{Gram} & -- &  &  & \bf 91.7 (91.0--92.4) & 2 & \bf 93.5 (93.1--93.9) & 2 \\
\hline
5-\textsc{Gram} & -- &  &  & 91.2 (90.4--91.9) & 3 & 93.4 (93.0--93.7) & 3 \\
\hline
4-\textsc{Gram} & -- &  &  & 89.6 (88.7--90.4) & 4 & 91.1 (90.6--91.6) & 4 \\
\hline
3-\textsc{Gram} & -- &  &  & 87.0 (86.1--87.8) & 5 & 87.0 (86.4--87.6) & 16 \\
\hline
2-\textsc{Gram} & -- &  &  & 83.3 (82.2--84.2) & 8 & 74.5 (73.6--75.3) & 24 \\
\hline
\textsc{Common-Suffix} & -- &  &  & 84.7 (83.6--85.6) & 6 & -- & -- \\
\hline
\textsc{Null} & -- &  &  & -- & -- & 68.9 (68.2--69.6) & 25 \\
\hline
\textsc{Random} & -- &  &  & 53.3 (51.7--54.7) & 26 & -- & -- \\
\hline
\hline

### Comparison of Prompts

In [5]:
multi_prompt_table_of_results(multi_prompts(results_t), multi_prompts(results_sc))

\begin{tabular}{|l|c|c|c|c|}
\hline
\bf Model & \bf \textsc{Basic} & \bf \textsc{More-Expl} & \bf \textsc{COT} & \bf \textsc{Red-Green} \\
\hline
\hline
\multicolumn{5}{|l|}{ \bf Sequence Completion} \\
\hline
gpt-4o-mini & \bf 72.4 (68.1--76.3) & 70.5 (66.4--74.6) & 58.0 (53.4--62.4) & 59.1 (54.9--63.2) \\
\hline
gpt-4o & 72.1 (65.9--78.2) & N/A & 67.4 (60.8--73.8) & \bf 74.4 (69.9--78.6) \\
\hline
claude-3.5 & N/A & N/A & \bf 84.0 (79.3--88.4) & 80.0 (74.9--85.2) \\
\hline
\hline
\multicolumn{5}{|l|}{ \bf Transducer} \\
\hline
gpt-4o-mini & \bf 79.8 (77.3--82.2) & 76.7 (74.2--79.3) & 65.2 (63.1--67.4) & 74.5 (72.0--77.0) \\
\hline
gpt-4o & \bf 83.7 (80.1--86.9) & 82.6 (79.1--85.9) & 67.8 (63.1--72.3) & 82.6 (78.8--86.3) \\
\hline
claude-3.5 & 86.9 (83.3--90.0) & \bf 87.1 (83.9--90.2) & 76.4 (72.9--79.9) & 82.9 (78.9--86.9) \\
\hline
\end{tabular}



### Commas

In [6]:
produce_table(
    {k: v for k, v in results_sc.items() if k in model_by_display_key},
    [display_prompt_sc(p) for p in ["Basic", "Basic-Commas"]],
)

\begin{tabular}{|r|c|c|}
\hline
Model & \textsc{Basic}$_S$ & \textsc{Basic-Commas}$_S$\\
\hline
\bf qwen-2.5-coder-7B &\bf 79.5 (78.4--80.5)&\bf 60.7 (59.3--62.1)\\
\hline
qwen-2.5-coder-instruct-7B &79.5 (78.3--80.5)&55.5 (54.0--56.9)\\
\hline
qwen-2.5-coder-instruct-32B &79.2 (78.0--80.3)&55.2 (53.7--56.7)\\
\hline
mistral-nemo-minitron-8B &78.7 (77.5--79.8)&59.3 (57.9--60.8)\\
\hline
codestral-22B &78.0 (76.8--79.1)&59.0 (57.5--60.3)\\
\hline
deepseek-coder-33b-instruct &76.7 (75.3--77.8)&54.9 (53.0--56.8)\\
\hline
mistral-nemo-base-12B &75.5 (74.3--76.6)&60.6 (59.1--62.2)\\
\hline
llama3.1-8B-Instruct &75.3 (74.0--76.6)&56.3 (54.4--58.1)\\
\hline
llama3-8B &73.8 (72.4--75.1)&61.5 (60.2--62.9)\\
\hline
starcoder2-15b &73.5 (72.0--74.7)&58.2 (56.7--59.8)\\
\hline
gemma-7b &72.6 (71.3--73.7)&54.0 (51.9--56.0)\\
\hline
gpt-4o-mini &72.4 (68.1--76.3)&64.1 (59.5--68.3)\\
\hline
mistral-nemo-instruct-12B &72.2 (70.9--73.4)&58.2 (56.4--59.8)\\
\hline
gpt-4o &72.1 (65.9--78.2)&66.8 (58.5--7